In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
# Download the MovieLens dataset
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip


In [ ]:
# Load the dataset
df = pd.read_csv('ml-latest-small/ratings.csv')

# Display dataset information
print("Dataset Sample:")
print(df.head())

# Check unique users and items
print("\nNumber of unique users:", df['userId'].nunique())
print("Number of unique movies:", df['movieId'].nunique())


In [ ]:
# Map user and movie IDs to indices
user_ids = df['userId'].unique()
movie_ids = df['movieId'].unique()

user_id_map = {id: index for index, id in enumerate(user_ids)}
movie_id_map = {id: index for index, id in enumerate(movie_ids)}

df['user_idx'] = df['userId'].map(user_id_map)
df['movie_idx'] = df['movieId'].map(movie_id_map)

# Split the data
X = df[['user_idx', 'movie_idx']].values
y = df['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nTraining Data Shape:", X_train.shape)
print("Test Data Shape:", X_test.shape)


In [ ]:
# Define input layers
user_input = Input(shape=(1,), name='user_input')
movie_input = Input(shape=(1,), name='movie_input')

# Define embedding layers
user_embedding = Embedding(input_dim=len(user_ids), output_dim=50, name='user_embedding')(user_input)
movie_embedding = Embedding(input_dim=len(movie_ids), output_dim=50, name='movie_embedding')(movie_input)

# Flatten embeddings
user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

# Concatenate user and movie embeddings
concat = Concatenate()([user_vec, movie_vec])

# Add dense layers
dense = Dense(128, activation='relu')(concat)
dense = Dropout(0.2)(dense)
dense = Dense(64, activation='relu')(dense)
output = Dense(1, activation='linear')(dense)

# Build the model
model = Model(inputs=[user_input, movie_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

# Display model summary
model.summary()


In [ ]:
# Train the model
history = model.fit(
    [X_train[:, 0], X_train[:, 1]],
    y_train,
    validation_data=([X_test[:, 0], X_test[:, 1]], y_test),
    epochs=10,
    batch_size=64
)

print("\nModel Training Complete!")


In [ ]:
# Evaluate model performance
loss, mae = model.evaluate([X_test[:, 0], X_test[:, 1]], y_test)
print(f"\nTest Loss: {loss:.4f}")
print(f"Test Mean Absolute Error: {mae:.4f}")


In [ ]:
# Function to recommend movies
def recommend_movies(user_id, model, user_id_map, movie_id_map, movie_ids, top_n=5):
    user_idx = user_id_map[user_id]
    movie_indices = [movie_id_map[movie] for movie in movie_ids]
    predictions = model.predict([np.array([user_idx] * len(movie_indices)), np.array(movie_indices)])
    recommendations = sorted(zip(movie_ids, predictions.flatten()), key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]

# Recommend movies for a specific user
user_id = 1  # Change to any user ID
recommendations = recommend_movies(user_id, model, user_id_map, movie_id_map, movie_ids)
print("\nTop Movie Recommendations:")
for movie, score in recommendations:
    print(f"Movie ID: {movie}, Predicted Rating: {score:.2f}")


In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
